In [2]:
#pip install matplotlib
#sqlite3 /home/synchronize_data/price_analytics/24.db.copied


In [ ]:
import matplotlib.pyplot as plt  
import sqlite3
import json

In [3]:
connection = sqlite3.connect('/home/synchronize_data/price_analytics/24.db.copied')
cursor = connection.cursor()

In [4]:
cursor.execute("""
SELECT 
    *
FROM copied
where 
    category = "Новостройки" 
    or category = "Реновация"
ORDER BY price desc
LIMIT 1
""")

In [5]:
flat = cursor.fetchall()

In [6]:
flat

[(1115,
  'Саввинская 27 27 6 236.6 Саввинская 27 55.730669,37.557942',
  809619049,
  809619049,
  '2022-12-05 13:53:38.523453',
  'Новостройки',
  'level.ru',
  'https://level.ru/sav27/apartment/4room/4-13/',
  236.6,
  None,
  None,
  None,
  None)]

Поля рекомендованные: id, "title", "price", "price_sale", date(datetime_create) as date, "brand", "apartment_area", "apartment_floor", "apartment_floors_total", "apartment_room", "apartment_ppm"
id: уникальный идентификатор квартиры в базе данных.
title: адрес квартиры.
price: цена квартиры.
price_sale: цена квартиры со скидкой (если есть).
date(datetime_create): дата и время создания записи о квартире.
brand: бренд или название компании, предлагающей эту квартиру.
apartment_area: площадь квартиры.
apartment_floor: номер этажа, на котором расположена квартира.
apartment_floors_total: общее количество этажей в доме.
apartment_room: количество комнат в квартире.
apartment_ppm: - цена за 1 м2

# Задачи:
sqlite3 /home/synchronize_data/price_analytics/24.db.copied\

.tables

SELECT * FROM copied limit 10;

#status-доступна есть https://api.gkosnova.tech/public/api/v1/building-objects/17/properties?detail=0

1. Выведите список новостроек в порядке убывания количества доступных к покупке квартир.

+ 2. Найдите среднюю цену квадратного метра по всем новостройкам

3. Выведите список новостроек в порядке убывания общего количества квартир.

+ 4. Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3

+ 5. Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.

+ 6. Найдите среднюю цен за квадратный метр в новостройках с количеством этажей больше 15.

+ 7. Найти все новостройки которые появились в 2023 году

8. Найти все новостройки которые имеют больше 50 квартир

+ 9. Найдите максимальную и минимальную цену среди всех новостроек

+ 10. Найти все новостройки в которых есть квартиры площадью больше 100 кв метров

+ 11. Найти новостройки котрые были построекны застройщщиком Донстрой

+ 12. Найти всех застройщиков которые построили более 5 новостроек в 2023 году

13. Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

In [8]:
# 1. Выведите список новостроек в порядке убывания количества доступных к покупке квартир.'''
# кол-во убывания - это по застройщику(фирме) фильтровать, сколько какая продает?

cursor.execute("""
SELECT DISTINCT
    title, price, price_sale, apartment_ppm, datetime_create
FROM copied
WHERE 
    category = 'Новостройки'
    AND (price IS NOT NULL OR price_sale IS NOT NULL)
    AND apartment_ppm IS NOT NULL
ORDER BY 
    price DESC   
LIMIT 10;
""")

cursor.fetchall()

#DISTINCT - удаление дубликатов

[('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 05:55:47.626915'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 06:32:24.745188'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 06:47:54.735178'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 08:13:04.621236'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 10:15:25.385630'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 12:32:35.235747'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959',
  807065785,
  807065785,
  2367457,
  '2024-01-20 12:52:17.874930'),
 ('Саввинская 27 27 2 340.9 Саввинская 27 55.730824,37.557959'

In [7]:
# 2. Найдите среднюю цену квадратного метра по всем новостройкам.

cursor.execute("""
SELECT 
    avg(apartment_ppm) as average_price_per_meter
FROM copied
where 
    category = "Новостройки" 
""") 
cursor.fetchall()


[(366205.35470005526,)]

In [9]:
#3. Выведите список новостроек в порядке убывания общего количества квартир.
#общее кол-во квартир в 1 новостройке - дублир кв ;-(? и адреса не будут совпадать, т к номера кв и площади в адресе указаны разные


In [15]:
%%sql
select title, count(*) as total_apartments
from copied
where category = 'Новостройки'
group by title
order by total_apartments desc
limit 40;

    

title,total_apartments
1 1 1 -1 13.25 0,10232
14 1 1 -2 13.25 0,7247
10 1 1 -1 13.25 0,4697
14 1 1 -1 13.25 0,4217
10 1 1 2 13.25 0,3839
10 1 1 1 13.25 0,3760
12 1 1 -2 13.25 0,3230
2 1 1 -4 13.25 0,2958
2 1 1 -3 13.25 0,2856
3 1 1 -1 13.3 0,2821


In [17]:
# 4. Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3. 

cursor.execute('''
select 
    round(avg(apartment_area),2) as average_area_flate
from copied
where
    category = "Новостройки" and apartment_room > 3
''')
cursor.fetchall()

[(119.86,)]

In [19]:
#5. Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.

cursor.execute('''
select 
    date(datetime_create), avg(apartment_ppm) as average_price_date
from copied
where
    category = "Новостройки" and date(datetime_create) = '2024-02-05'
''')
cursor.fetchall()

[('2024-02-05', 600898.7923076923)]

In [20]:
# 6. Найдите среднюю цен за квадратный метр в новостройках с количеством этажей больше 15.
cursor.execute('''
select
    avg(apartment_ppm) as average_price_per_meter
from copied
where category = "Новостройки" and apartment_floor > 15
''')
cursor.fetchall()

[(418851.49659148,)]

In [21]:
#7. Найти все новостройки которые появились в 2023 году

cursor.execute('''
select
    title, date(datetime_create) as date
from copied
where category = "Новостройки" and date like '2023%'
limit 10
''')
cursor.fetchall()


[('Павелецкая Сити 9 2 23.8 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035',
  '2023-01-01'),
 ('Селигерская 1 3 17.8 пр-д Ильменский, вл. 12 55.859441,37.545155',
  '2023-01-01'),
 ('Стрешнево Север 2 45.1 Волоколамское ш., д. 81, корп. 2 55.82495356307498,37.411886931513436',
  '2023-01-01'),
 ('Южнопортовая 6 8 55.3 Г. Москва, ул. Южнопортовая, вл.28-28а 55.705727,37.696445',
  '2023-01-01'),
 ('Южнопортовая 1 39 78.3 Г. Москва, ул. Южнопортовая, вл.28-28а 55.705727,37.696445',
  '2023-01-01'),
 ('Стрешнево Восток 24 155.2 Волоколамское ш., д. 81, корп. 2 55.82495356307498,37.411886931513436',
  '2023-01-01'),
 ('Мичуринский 6 15 112.3 Г. Москва, Озерная ул., вл. 7 55.687726,37.471704',
  '2023-01-01'),
 ('Мичуринский 5 36 178.0 Г. Москва, Озерная ул., вл. 7 55.687726,37.471704',
  '2023-01-01'),
 ('Мичуринский 5 36 245.5 Г. Москва, Озерная ул., вл. 7 55.687726,37.471704',
  '2023-01-01'),
 ('Павелецкая Сити 6 9 23.6 ул. Дубининская, д. 59-69 55.71962004653103,37.6372

8. Найти все новостройки которые имеют больше 50 квартир

In [ ]:
# %%sql 
# select
# from codied
# where > 50

9. Найдите максимальную и минимальную цену среди всех новостроек 

In [48]:
%%sql
select max(price), min(price) 
from copied 
where category = "Новостройки" and price != 0


max(price),min(price)
809619049,551000


In [29]:
#10. Найти все новостройки в которых есть квартиры площадью больше 100 кв метров

cursor.execute('''
select id, title
from copied
where category = 'Новостройки' and apartment_area > 100
limit 15
''')
cursor.fetchall()

[(649, 'Саввинская 27 27 2 137.5 Саввинская 27 55.730669,37.557942'),
 (651, 'Саввинская 27 27 3 139.5 Саввинская 27 55.730669,37.557942'),
 (653, 'Саввинская 17 17 4 110.5  55.734600731695195,37.565715889000494'),
 (655, 'Саввинская 27 27 6 189.9 Саввинская 27 55.730669,37.557942'),
 (662, 'Мичуринский 6 15 112.3 Озерная ул., вл. 7 55.687726,37.471704'),
 (663, 'Мичуринский 5 36 178.0 Озерная ул., вл. 7 55.687726,37.471704'),
 (664, 'Мичуринский 5 36 245.5 Озерная ул., вл. 7 55.687726,37.471704'),
 (670, 'Мичуринский 6 18 112.3 Озерная ул., вл. 7 55.687726,37.471704'),
 (671, 'Мичуринский 5 36 185.9 Озерная ул., вл. 7 55.687726,37.471704'),
 (672, 'Мичуринский 4 44 249.1 Озерная ул., вл. 7 55.687726,37.471704'),
 (678, 'Мичуринский 6 19 112.3 Озерная ул., вл. 7 55.687726,37.471704'),
 (679, 'Мичуринский 4 44 245.9 Озерная ул., вл. 7 55.687726,37.471704'),
 (684, 'Мичуринский 4 43 122.6 Озерная ул., вл. 7 55.687726,37.471704'),
 (685,
  'Стрешнево Север 25 227.9 Волоколамское ш., д. 81

11. Найти новостройки котрые были построекны застройщщиком Донстрой (# можно было title like "%donstroy%")

In [47]:
%%sql
select id, title, brand, date(datetime_create) as date
from copied
where category = 'Новостройки' and brand = 'donstroy.moscow'
limit 15

id,title,brand,date
29629804,"Символ Независимость Корпус 19, 2 секц 4 27.1",donstroy.moscow,2024-02-09
29629805,"Символ Независимость Корпус 19, 2 секц 3 28.3",donstroy.moscow,2024-02-09
29629806,"Символ Независимость Корпус 19, 2 секц 4 27.1",donstroy.moscow,2024-02-09
29629807,"Символ Независимость Корпус 19, 2 секц 3 29.9",donstroy.moscow,2024-02-09
29629808,"Символ Независимость Корпус 19, 2 секц 5 27.1",donstroy.moscow,2024-02-09
29629809,"Символ Независимость Корпус 19, 2 секц 5 29.4",donstroy.moscow,2024-02-09
29629810,"Символ Независимость Корпус 19, 2 секц 5 27.1",donstroy.moscow,2024-02-09
29629811,"Символ Независимость Корпус 19, 2 секц 5 28.3",donstroy.moscow,2024-02-09
29629812,"Символ Независимость Корпус 19, 2 секц 4 29.9",donstroy.moscow,2024-02-09
29629813,"Символ Независимость Корпус 19, 2 секц 5 29.9",donstroy.moscow,2024-02-09


12. Найти всех застройщиков которые построили более 5 новостроек в 2023 году

In [46]:
%%sql
SELECT brand, count(*)
FROM copied
WHERE category = 'Новостройки' 
    AND datetime_create LIKE '2023%' 
GROUP BY brand
HAVING count(*) > 5


brand,count(*)
Capital Group,3932
level.ru,346656
Галс-Девелопмент,872


In [38]:
#12. Найти всех застройщиков которые построили более 5 новостроек в 2023 году
%sql SELECT brand, count(*) FROM copied WHERE category = 'Новостройки' and datetime_create LIKE '2023%' GROUP BY brand HAVING count(*) > 5

brand,count(*)
Capital Group,3932
level.ru,346656
Галс-Девелопмент,872


13. Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

In [23]:
# cursor.execute("""
# SELECT 
    
# FROM copied
# where 
#     category = "Новостройки" 
# """)
# cursor.fetchall()

In [27]:
#для себя посмотреть вывод табл кв за 2024 - столбцы
%sql select * from copied where datetime_create like '2024%' limit 10


id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
26513533,"кв. 49_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 8 57.4 Вилиса Лациса ул., д. 38, к. 1, 2",14200760,14200760,2024-01-20 05:33:23.754343,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19151571,57.4,8,17,2,247400
26513534,"кв. 88_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 12 57.2 Вилиса Лациса ул., д. 38, к. 1, 2",14294280,14294280,2024-01-20 05:33:23.763560,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19151613,57.2,12,17,2,249900
26513535,"кв. 96_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 13 57.4 Вилиса Лациса ул., д. 38, к. 1, 2",14344260,14344260,2024-01-20 05:33:23.771348,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146214,57.4,13,17,2,249900
26513536,"кв. 105_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 15 57.6 Вилиса Лациса ул., д. 38, к. 1, 2",14394240,14394240,2024-01-20 05:33:23.779571,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146237,57.6,15,17,2,249900
26513537,"кв. 114_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 16 68.7 Вилиса Лациса ул., д. 38, к. 1, 2",16117020,16117020,2024-01-20 05:33:23.831949,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146255,68.7,16,17,3,234600
26513538,"кв. 123_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 2 38.2 Вилиса Лациса ул., д. 38, к. 1, 2",9733360,9733360,2024-01-20 05:33:23.841861,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146284,38.2,2,17,1,254800
26513539,"кв. 243_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 17 37.9 Вилиса Лациса ул., д. 38, к. 1, 2",10058660,10058660,2024-01-20 05:33:23.850827,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146304,37.9,17,17,1,265400
26513540,"кв. 172_Нижегородская ул., д. 74 nizhegorodskaya-ul-d-74 2 73.8 Нижегородская ул., д. 74",19726740,19726740,2024-01-20 05:33:23.904525,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19416185,73.8,2,13,3,267300
26513541,"кв. 176_Нижегородская ул., д. 74 nizhegorodskaya-ul-d-74 2 74.4 Нижегородская ул., д. 74",19887120,19887120,2024-01-20 05:33:23.912713,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19416224,74.4,2,13,3,267300
26513542,"кв. 233_Нижегородская ул., д. 74 nizhegorodskaya-ul-d-74 2 76 Нижегородская ул., д. 74",20314800,20314800,2024-01-20 05:33:23.919829,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19416272,76,2,13,3,267300


In [2]:
# ставлю релоад вместо лоад чтобы можно было перезапускать весь ноутбук. комментарий на отдельной строчке, по другому не надо
#pip install ipython-sql

In [10]:
%reload_ext sql

In [11]:
%config SqlMagic.displaycon = False


In [12]:
%config SqlMagic.feedback = False

In [13]:
%sql sqlite:////home/synchronize_data/price_analytics/24.db.copied
#%sql sqlite:///24.db.copied

In [14]:
#далее можно вот так писать sql запросы и даже передавать результаты их выполнения в переменные - смотрите по ссылке вверху этого ноутбука
%sql SELECT * from copied WHERE id<600

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
598,"Причальный Bосток 29 65.3 Причальный пр., д. 8 55.76018,37.517482",37527037,26268925,2022-12-05 13:38:36.641439,Новостройки,level.ru,url_example,None,None,None,None,None
599,"Саввинская 17 17 4 82.5 55.734600731695195,37.565715889000494",202359478,202359478,2022-12-05 13:38:36.652957,Новостройки,level.ru,url_example,None,None,None,None,None


# Задачи с обогащением:


1. Построить месячный график количества продаж новостроек за последний год.

2. Выведите список новостроек в западном районе.

3. Найти все новостройки в ЗАО и СЗАО которые имеют больше 10 этажей

4. Найти самый популярный размер квартир в каждом округе Москвы, и посчитать количеств продаж каждого размера ( сейчас за размер принимаем количество комнат)

5. Найти все новостройки в пределах заданного радиуса от опеределнного места. И определить расстояние от каждой новостройки до заданного места, распределив их по возрастанию расстояния до заданного места.

6. Построить график средней стоимости квадратного метра по месяцам, начиная с q2 2023 по q2 2024

7. Найти все новостройки в которых есть квартиры с видом на определенный объект ( например парк, реку, останкинскую телебашню, москва сити или другую достопримечательность).

8. Показать информацию о количестве квартир с таким видом их средние площади и стоимости.

9. Вывести список жилых комплексов с наибольшей разницей между стоимостью квартир в них и среденей стоимостью квартир в районе. Рассчитать процентное отклонение цены каждой квартиры от средней цены по району.

10. Начиная с 2023 года, построить несколько графиков по месяцам : стоимости кв метра квартир, курса доллара(сколько рублей за один доллар), ключевой ставки, уровень безработиц, численность населения в городе, среднее количество детей в семье, количество сделок по ипотеке, количество сделок по льготным ипотечным программам,

11. Найти все новостройки информация о которых появилась в 2023 году имеют рейтинг выше среднего по городу. Вывести список этих новостроек, среднее, минимальное и максимальное количество этажей в них, в разрезе типа квартир ( тип это количество комнат ) с количеством квартир.

12. Вывести весь жк квартиры в которых были введены в эксплуатацию с чистовой отделкой в 2024 году

13. вывести список жк бизнес класса в ВАО города Москвы района Измайлово, ключи от которых владельцы получили в 2023

14. Выведите жк где соотношение количества квартир к количеств парковочных мест меньше 2.5

15. Введите количество квартир в кирпичных домах города Москвы

16. Определите среднее время оживания лита в домах застройщика level

17. Выведите список жк от которых до ближайшей станции метро 10 минут пешком

18. Выведите общее число объектов застройщиков которые больше 20 лет на рынке, и при качестве объектов выше среднего.



In [ ]:
#1. Построить месячный график количества продаж новостроек за последний год.

# Задачи на подумать и предложить Выдвинуть 10 гипотез и проверить их на основе данных ( если данных не хватает то добавить данными из интернета)


In [ ]:
Гипотеза 1 - стоимость квартиры зависит от площади

Гипотеза 2 - стоимость квартиры зависит от расстояния до центра города

Гипотеза 3 -